In [ ]:
import os

import pymongo
from bson.json_util import dumps, loads
from bson.objectid import ObjectId

from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.common.exceptions import TimeoutException

import pandas as pd

import re
from pprint import pprint
from datetime import datetime as dt
import time
import isodate

import sys
# sys.path.append('C:\\Users\\soria\\Documents\\WashUDataDocuments\\HwActivites\\YouTube_Project\\')

import pandas as pd
import json
import os
from pprint import pprint
import random

In [ ]:
os.getcwd()

In [ ]:
executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
# browser = Browser('chrome', **executable_path)
driver = webdriver.Chrome()

In [ ]:
url = 'https://www.seriouseats.com/sitemap'
driver.get(url)

In [ ]:
html = driver.page_source
se_sm_soup = bs(html, 'html.parser')

In [ ]:
se_recipe_links_dict = {}
topic_anchors = se_sm_soup.find_all('a', href=re.compile('topics'))

In [ ]:
se_recipe_links_dict['ingredient'] = []
se_recipe_links_dict['method'] = []
se_recipe_links_dict['meal'] = []
se_recipe_links_dict['cuisine'] = []

for i, anchor in enumerate(topic_anchors):
#     print(f"{i}: {anchor.text.strip()}")
#     print(anchor['href'])
    link = anchor['href']
    if 'ingredient' in link:
        this_dict = {'topic': anchor.text.strip(), 'ingredient_url(s)': [link]}
        se_recipe_links_dict['ingredient'].append(this_dict)
    if 'method' in link:
        this_dict = {'topic': anchor.text.strip(), 'method_url(s)': [link]}
        se_recipe_links_dict['method'].append(this_dict)
    if 'meal' in link:
        this_dict = {'topic': anchor.text.strip(), 'meal_url(s)': [link]}
        se_recipe_links_dict['meal'].append(this_dict)
    if 'cuisine' in link:
        this_dict = {'topic': anchor.text.strip(), 'cuisine_url(s)': [link]}
        se_recipe_links_dict['cuisine'].append(this_dict)

In [ ]:
se_recipe_links_dict['cuisine'] = se_recipe_links_dict['cuisine'][1:]
se_recipe_links_dict['ingredient'] = se_recipe_links_dict['ingredient'][0:2] + se_recipe_links_dict['ingredient'][3:]
se_recipe_links_dict['method'] = se_recipe_links_dict['method'][0:2] + se_recipe_links_dict['method'][3:]
pprint(se_recipe_links_dict)

In [ ]:
def href_filt(href):
    return re.compile('recipe').search(href) and re.compile('https').search(href) and not re.compile('topics').search(href)

In [ ]:
test = {'cuisine': se_recipe_links_dict['cuisine']}
for topic, _list in se_recipe_links_dict.items():
    for _dict in _list:
        _dict.setdefault('recipes', [])
        string = topic + '_url(s)'
        print(f"string: {string}")
        for i, u in enumerate(_dict[string]):
            cfTopic = re.sub(r'\s', '_', _dict['topic'])
            cfTopic = re.sub(r'/', '_', cfTopic)
            print(cfTopic)
            cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\recipe_urls\\{cfTopic}_{i}.txt"
            print(f"cfp: {cacheFilePath}")
            if os.path.isfile(cacheFilePath):
                with open(cacheFilePath, encoding='utf-8') as cacheFile:
                    html = cacheFile.read()
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        _dict['recipes'].append(this_dict)
            else:
                url = u
                print(f"url: {url}")
                time.sleep(random.uniform(0,3))
                driver.get(url)
                html = driver.page_source
                with open(cacheFilePath, "w", encoding='utf-8') as cacheFile:
                    cacheFile.write(html)        

                se_recipe_soup = bs(html, 'html.parser')
                content = se_recipe_soup.find('div', class_='content-main')
                recipe_anchors = content.find_all('a', href=href_filt)
                recipe_anchors = recipe_anchors[2:]
                for anchor in recipe_anchors[::2]:
                    this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                    _dict['recipes'].append(this_dict)
                if se_recipe_soup.find('a', text=(re.compile('Next'))):
                    time.sleep(random.uniform(0,5))
                    url = se_recipe_soup.find('a', text=(re.compile('Next')))['href']
                    _dict[string].append(url)
                    driver.get(url)
                    html = driver.page_source
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        _dict['recipes'].append(this_dict)

In [ ]:
# pprint(se_recipe_links_dict)
# se_recipe_links_dict.keys()
sys.getsizeof(se_recipe_links_dict)

In [ ]:
cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\recipe_urls"
filess = []
for root, dirs, files in os.walk(cacheFilePath):

    for file in files:
        filess.append(file)


In [ ]:
for i, file in enumerate(filess):
    print(f"processing:{i} of {len(filess)}: {file}")
    for topic, _list in se_recipe_links_dict.items():
        for _dict in _list:
            _dict.setdefault('recipes', [])
            cfTopic = re.sub(r'\s', '_', _dict['topic'])
            cfTopic = re.sub(r'/', '_', cfTopic)
            if cfTopic in file:
                cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\recipe_urls\\{file}"
                with open(cacheFilePath, encoding='utf-8') as cacheFile:
                    html = cacheFile.read()
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    recipe_anchors = recipe_anchors[2:]
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        _dict['recipes'].append(this_dict)

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats_test


In [ ]:
collection = mongoDb.se_recipe_just_url_test
collection.insert_one(se_recipe_links_dict)

In [ ]:
collection = mongoDb.se_recipe_just_url_test
url_dict = collection.find_one({}, {'_id':False})

In [ ]:
collection = mongoDb.se_recipe_just_url_test
unupdated_dict = collection.find_one({}, {"recipes.updated":{'$exists': False}})
unupdated_dict

In [ ]:
for topic, topic_list in url_dict.items():
    for index, subtopic_dict in enumerate(topic_list):
        for _, _ in subtopic_dict.items():
            for subindex, recipe_dict in enumerate(subtopic_dict['recipes']):
                print(recipe_dict['recipe_title'])
                

In [ ]:
b.nested.findOne({"level1":{"$elemMatch":{"$in":['item00']}} })

In [ ]:
# grab full list of recipe urls

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats_test

collection = mongoDb.se_recipe_just_url_test
url_dict = collection.find_one({}, {'_id':False})

In [ ]:
# create new database with collection names spread out

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats_test2

for topic, topic_list in url_dict.items():
    print(f'topic: {topic}')
    for index, subtopic_dict in enumerate(topic_list):
        collection = mongoDb[subtopic_dict['topic']]
        collection.create_index('recipe_url', unique=True)
        for _, _ in subtopic_dict.items():
            for subindex, recipe_dict in enumerate(subtopic_dict['recipes']):
                url = recipe['recipe_url']
                print(f"{topic}: {i} of {len(topic_dict['recipes'])} => {url}")
                title = recipe['recipe_title']
                this_recipe = {
                    'topic': subtopic_dict['topic'], 'source': 'serious_eats', 
                    'recipe_title': recipe['recipe_title'], 'recipe_url': recipe['recipe_url']
                }
                try:
                    collection.insert_one(this_recipe)
                except pymongo.errors.DuplicateKeyError:
                    continue

In [ ]:
# make list from collection names minus full url list collection

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats_test2

collection_names = mongoDb.list_collection_names()
coll_names = []
for i, coll in enumerate(collection_names):
    print(i, coll)
    coll_names.append(coll)

coll_names = coll_names[:105] + coll_names[106:107]
coll_names

In [ ]:
for i, coll in enumerate(coll_names):
    print(i, coll)

In [ ]:
# set updated timestamp to url collection

for topic, topic_list in url_dict.items():
    for index, subtopic_dict in enumerate(topic_list):
        for _, _ in subtopic_dict.items():
            for subindex, recipe_dict in enumerate(subtopic_dict['recipes']):
                this_url = recipe_dict['recipe_url']
                mongoDb = client.serious_eats_test
                collection = mongoDb[subtopic_dict['topic']]
                if collection.find_one({'recipe_url':this_url}):
#                     collection = mongoDb.se_recipe_just_url
                    this_recipe = {
                        'topic': subtopic_dict['topic'], 'source': 'serious_eats', 
                        'recipe_title': recipe_dict['recipe_title'],
                        'recipe_url': recipe_dict['recipe_url'],
                        'updated': dt.now()
                    }
                    this_query = f"{subindex}"
                    print(f"processing: {subtopic_dict['topic']} => {this_query}")
                    mongoDb = client.serious_eats_test2
                    collection = mongoDb[subtopic_dict['topic']]
                    collection.update_one({'recipe_url': this_url}, {"$set": this_recipe}, upsert=True)


In [ ]:
# create dictionary of unupdated ids

mongoDb = client.serious_eats_test2
unupdated = {}
for coll in coll_names:
    collection = mongoDb[coll]
    unupdated[coll] = []
    r = collection.find({"updated":{'$exists': False}}, {})
    for i in r:
        unupdated[coll].append(i['_id'])


In [ ]:
unupdated

In [ ]:
# create dictionary of unupdated recipe urls from ids


unupdated_url_dict = {}
for coll, obj_ids in unupdated.items():
    collection = mongoDb[coll]
    unupdated_url_dict[coll] = []
    for _id in obj_ids:
        r = collection.find_one({'_id': ObjectId(_id)})
        print(f"processing:{coll}{_id}")
        topic = r['topic']
        url = r['recipe_url']
        title = r['recipe_title']
        this_dict = {'topic': topic, 'recipe_url': url, 'recipe_title': title}
        unupdated_url_dict[coll].append(this_dict)


In [ ]:
for coll, recipe_list in unupdated_url_dict.items():
    collection = mongoDb[coll]
    for recipe_dict in recipe_list:
        recipe_dict.setdefault('recipes', [])
        for i, u in enumerate(recipe_dict):
            cfTopic = re.sub(r'\s', '_', recipe_dict['topic'])
            cfTopic = re.sub(r'/', '_', cfTopic)
            print(cfTopic)
            cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\recipe_urls\\{cfTopic}_{i}.txt"
            print(f"cfp: {cacheFilePath}")
            if os.path.isfile(cacheFilePath):
                with open(cacheFilePath, encoding='utf-8') as cacheFile:
                    html = cacheFile.read()
                    se_recipe_soup = bs(html, 'html.parser')
                    content = se_recipe_soup.find('div', class_='content-main')
                    recipe_anchors = content.find_all('a', href=href_filt)
                    for anchor in recipe_anchors[::2]:
                        this_dict = {'recipe_title': anchor.text, 'recipe_url': anchor['href']}
                        recipe_dict['recipes'].append(this_dict)
                        collection.update_one({'recipe_url':recipe_dict['recipe_url']}, {'$set':{'recipes':anchor['href']}}, upsert=True)

In [ ]:
## use if se_recipe_links_dict is created from scratch

for topic, topic_list in url_dict.items():
    print(f'topic: {topic}')
    for topic_dict in topic_list:
        collection = mongoDb[topic_dict['topic']]
        collection.create_index('recipe_url', unique=True)
        for i, recipe in enumerate(topic_dict['recipes']):
            url = recipe['recipe_url']
            print(f"{topic}: {i} of {len(topic_dict['recipes'])} => {url}")
            cfTitle = re.sub(r"\s|/|\.|,|'|-|\?|\||<|>|\\|\*|:", '_', recipe['recipe_title'])
            cfTitle = re.sub(r'"', '', cfTitle)
            cfTitle = re.sub(r'__', '_', cfTitle)
            if cfTitle[0:1] == "_":
                cfTitle = cfTitle[1:]
            if cfTitle[-1:] == "_":
                cfTitle = cfTitle[:-1]
            cfTitle = cfTitle[:32]
            print(f"cfTitle: {cfTitle}")
            cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\{cfTitle}.txt"
            print(f"cfp: {cacheFilePath}")
            
            if os.path.isfile(cacheFilePath):
                with open(cacheFilePath, encoding='utf-8') as cacheFile:
                    html = cacheFile.read()
            else:
                driver.get(url)
                html = driver.page_source          
                with open(cacheFilePath, 'w', encoding='utf-8') as cacheFile:
                    cacheFile.write(html)
            se_soup = bs(html, 'html.parser')
            script_divs = se_soup.find_all('script', {'type': 'application/ld+json'})
#             recipe['category_trees'] = []
#             recipe['recipe'] = []
            category_trees = []
            for index, div in enumerate(script_divs):
                try:
                    j = json.loads(div.text, strict=False)

                    recipe.setdefault('category_trees', [])
                    recipe.setdefault('recipe', {})

                    if j['@type'] == 'BreadcrumbList':
                        category_tree = []
                        for item in j['itemListElement']:
                            category_tree.append(item['item']['name'])
    #                     recipe['category_trees'].append(category_tree)
                        category_trees.append(category_tree)

                    if j['@type'] == 'Recipe':
                        try:
                            rating_count = j['aggregateRating']['ratingCount']
                        except:
                            rating_count = False
                        try:
                            rating_value = j['aggregateRating']['ratingValue']
                        except:
                            rating_value = False
                        try:
                            author = j['author']['name']
                        except:
                            author = False
                        try:
                            job_title = j['author']['jobTitle']
                        except:
                            job_title = False
                        try:
                            description = j['description']
                        except:
                            description = False
                        try:
                            keywords = j['keywords']
                        except:
                            keywords = False
                        try: 
                            name = j['name']
                        except:
                            name = False
                        try:
                            categories = j['recipeCategory']
                        except:
                            categories = False
                        try:
                            cuisine = j['recipeCuisine']
                        except:
                            cuisine = False
                        try:
                            ingredients = j['recipeIngredient']
                        except:
                            ingredients = False
                        try:   
                            steps = {}
                            for i, step in enumerate(j['recipeInstructions']):
                                this = 'step_' + str(i+1)
                                steps[this] = step['text']
                        except:
                            steps = False
                        try:
                            recipe_yield = j['recipeYield']
                        except:
                            recipe_yield = False
                        try:
                            time = str(isodate.parse_duration(j['totalTime']).total_seconds()/60)
                        except:
                            time = False
    #                     recipe['recipe'] = {
                        json_recipe = {
                            'name': name, 'rating_count': rating_count, 'rating_value': rating_value, 'author': author,
                            'job_title': job_title, 'description': description, 'keywords': keywords, 'categories': categories,
                            'cuisine': cuisine, 'ingredients': ingredients, 'steps': steps, 'recipe_yield': recipe_yield, 'time':time
                        }
                except:
                    print(f"json error on:{url} - script_div{index}")
                    continue
            this_recipe = {
                'topic': topic_dict['topic'], 'source': 'serious_eats', 
                'recipe_title': recipe['recipe_title'], 'recipe_url': recipe['recipe_url'], 
#                 'category_trees': recipe['category_trees'], 'recipe': recipe['recipe'], 
                'category_trees': recipe['category_trees'], 'recipe': json_recipe, 
                'uploaded': dt.now()
            }
            try:
                collection.insert_one(this_recipe)
            except pymongo.errors.DuplicateKeyError:
                continue
#             collection.update_one({}, {'$set': collection.this_recipe}, upsert=True)


In [ ]:
cfTitle = re.sub(r"\s|/|\.|,|'|-", '_', 'C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\Weeknight_Turkey_"Bolognese".txt')
cfTitle = re.sub(r'"', '', cfTitle)
cfTitle = re.sub(r'__', '_', cfTitle)
cfTitle

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
mongoDb = client.serious_eats_test_2


In [ ]:
# se_recipe_links_dict.pop('_id')
for topic, _list in se_recipe_links_dict.items():
    collection = mongoDb[topic]
    collection.insert_many(_list)

In [ ]:
## use if pulled from mongo

for topic, topic_dict in se_recipe_links_dict.items():
    print(f'topic: {topic}')
    for i, recipe in enumerate(topic_dict['recipes']):
        url = recipe['recipe_url']
        print(f"{topic}: {i} of {len(topic_dict['recipes'])} => {url}")
        driver.get(url)
        html = driver.page_source
        se_soup = bs(html, 'html.parser')
        script_divs = se_soup.find_all('script', {'type': 'application/ld+json'})
        for div in script_divs:
            j = json.loads(div.text, strict=False)

            recipe.setdefault('category_trees', [])
            recipe.setdefault('recipe', {})

            if j['@type'] == 'BreadcrumbList':
                category_tree = []
                for item in j['itemListElement']:
                    category_tree.append(item['item']['name'])
                recipe['category_trees'].append(category_tree)
            if j['@type'] == 'Recipe':
                try:
                    rating_count = j['aggregateRating']['ratingCount']
                except:
                    rating_count = False
                try:
                    rating_value = j['aggregateRating']['ratingValue']
                except:
                    rating_value = False
                try:
                    author = j['author']['name']
                except:
                    author = False
                try:
                    job_title = j['author']['jobTitle']
                except:
                    job_title = False
                try:
                    description = j['description']
                except:
                    description = False
                try:
                    keywords = j['keywords']
                except:
                    keywords = False
                try: 
                    name = j['name']
                except:
                    name = False
                try:
                    categories = j['recipeCategory']
                except:
                    categories = False
                try:
                    cuisine = j['recipeCuisine']
                except:
                    cuisine = False
                try:
                    ingredients = j['recipeIngredient']
                except:
                    ingredients = False
                try:   
                    steps = {}
                    for i, step in enumerate(j['recipeInstructions']):
                        this = 'step_' + str(i+1)
                        steps[this] = step['text']
                except:
                    steps = False
                try:
                    recipe_yield = j['recipeYield']
                except:
                    recipe_yield = False
                try:
                    time = str(isodate.parse_duration(j['totalTime']).total_seconds()/60)
                except:
                    time = False
                recipe['recipe'] = {
                    'name': name, 'rating_count': rating_count, 'rating_value': rating_value, 'author': author,
                    'job_title': job_title, 'description': description, 'keywords': keywords, 'categories': categories,
                    'cuisine': cuisine, 'ingredients': ingredients, 'steps': steps, 'recipe_yield': recipe_yield, 'time':time
                }



In [ ]:
## use for unupdated dict

mongoDb = client.serious_eats_test

for coll, recipe_list in unupdated_url_dict.items():
    
    collection = mongoDb[coll]
    collection.create_index('recipe_url', unique=True)
    for i, recipe_dict in enumerate(recipe_list):
        url = recipe_dict['recipe_url']
        print(f"{coll}: {i} of {len(recipe_list)} => {url}")
        cfTitle = re.sub(r"\s|/|\.|,|'|-|\?|\||<|>|\\|\*|:", '_', recipe_dict['recipe_title'])
        cfTitle = re.sub(r'"', '', cfTitle)
        cfTitle = re.sub(r'__', '_', cfTitle)
        if cfTitle[0:1] == "_":
            cfTitle = cfTitle[1:]
        if cfTitle[-1:] == "_":
            cfTitle = cfTitle[:-1]
        cfTitle = cfTitle[:32]
        print(f"cfTitle: {cfTitle}")
        cacheFilePath = f"C:\\Users\\soria\\Documents\\resources\\caches\\serious_eats_html_caches\\{cfTitle}.txt"
        print(f"cfp: {cacheFilePath}")

        if os.path.isfile(cacheFilePath):
            with open(cacheFilePath, encoding='utf-8') as cacheFile:
                html = cacheFile.read()
        else:
            driver.get(url)
            wait = WebDriverWait(driver, 10)
            wait.until(
                EC.presence_of_element_located((By.XPATH, "//script[@type='application/ld+json']")))
            html = driver.page_source          
            with open(cacheFilePath, 'w', encoding='utf-8') as cacheFile:
                cacheFile.write(html)
        se_soup = bs(html, 'html.parser')
        script_divs = se_soup.find_all('script', {'type': 'application/ld+json'})
#             recipe_dict['category_trees'] = []
#             recipe_dict['recipe'] = []
        category_trees = []
        for index, div in enumerate(script_divs):
            try:
                j = json.loads(div.text, strict=False)

                recipe_dict.setdefault('category_trees', [])
                recipe_dict.setdefault('recipe', {})

                if j['@type'] == 'BreadcrumbList':
                    category_tree = []
                    for item in j['itemListElement']:
                        category_tree.append(item['item']['name'])
    #                     recipe_dict['category_trees'].append(category_tree)
                    category_trees.append(category_tree)

                if j['@type'] == 'Recipe':
                    try:
                        rating_count = j['aggregateRating']['ratingCount']
                    except:
                        rating_count = False
                    try:
                        rating_value = j['aggregateRating']['ratingValue']
                    except:
                        rating_value = False
                    try:
                        author = j['author']['name']
                    except:
                        author = False
                    try:
                        job_title = j['author']['jobTitle']
                    except:
                        job_title = False
                    try:
                        description = j['description']
                    except:
                        description = False
                    try:
                        keywords = j['keywords']
                    except:
                        keywords = False
                    try: 
                        name = j['name']
                    except:
                        name = False
                    try:
                        categories = j['recipeCategory']
                    except:
                        categories = False
                    try:
                        cuisine = j['recipeCuisine']
                    except:
                        cuisine = False
                    try:
                        ingredients = j['recipeIngredient']
                    except:
                        ingredients = False
                    try:   
                        steps = {}
                        for i, step in enumerate(j['recipeInstructions']):
                            this = 'step_' + str(i+1)
                            steps[this] = step['text']
                    except:
                        steps = False
                    try:
                        recipe_yield = j['recipeYield']
                    except:
                        recipe_yield = False
                    try:
                        time = str(isodate.parse_duration(j['totalTime']).total_seconds()/60)
                    except:
                        time = False
    #                     recipe_dict['recipe'] = {
                    json_recipe = {
                        'name': name, 'rating_count': rating_count, 'rating_value': rating_value, 'author': author,
                        'job_title': job_title, 'description': description, 'keywords': keywords, 'categories': categories,
                        'cuisine': cuisine, 'ingredients': ingredients, 'steps': steps, 'recipe_yield': recipe_yield, 'time':time
                    }
            except ValueError:
                print('==================')
                print(f"      json error on:{url} - script_div{index}")
                print('==================')
                json_recipe = ''
                continue
        this_recipe = {
            'topic': recipe_dict['topic'], 'source': 'serious_eats', 
            'recipe_title': recipe_dict['recipe_title'], 'recipe_url': recipe_dict['recipe_url'], 
#                 'category_trees': recipe_dict['category_trees'], 'recipe': recipe_dict['recipe'], 
            'category_trees': category_trees, 'recipe': json_recipe, 
            'uploaded': dt.now()
        }
        try:
            collection.insert_one(this_recipe)
        except pymongo.errors.DuplicateKeyError:
            continue
#             collection.update_one({}, {'$set': collection.this_recipe}, upsert=True)
